## Importações

In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
from pandas.util.testing import assert_frame_equal #import alterado

from tqdm import tqdm_notebook, tqdm
from collections import deque

<ipython-input-1-7eea41f82daf>:8: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal #import alterado


In [2]:
tf.__version__

'2.3.0'

In [3]:
class IA_Trader():
    """
    state_size = estados que estão vindo do ambiente, por exemplo os sensores de
     um carro autônomo. Seria a camada de entrada da rede neural.

    action_space = indica o numero de ações que o algoritmo pode tomar, ou seja,
    suas escolhas. Comprar, vender, ou não fazer nada.

    model_name = Nome do modelo de aprendizagem.
    """
    def __init__(self, state_size, action_space = 3, model_name="IATrader"):
        self.state_size = state_size
        self.action_space = action_space
        # memmoria de 2000 ações, ou seja, o modelo não precisa ficar treinando algumas etapas repetitivas
        self.memory = deque(maxlen = 2000)
        #memoria da experiencia de replay
        self.model_name = model_name

        #gama da equação de bellmam = fator de desconto
        self.gamma = 0.95

        #indica se as ações serão tomadas randomicamente ou pela rede neural. Sendo 1.0 = 100% randomico
        self.epsilon = 1.0

        #porcentegem do modelo tomar uma ação randomica, ou seja, com o valor de 0.1 o 
        #agente tem 10% de chance de tomar uma ação randomica e fazer ele explorar mais o ambiente.
        self.epsilon_final = 0.01

        #variavel responsável por decrementar o valor de epsilon, para poder fazer o ajuste do peso da rede.
        self.epsilon_decay = 0.995

        #criando o modelo quando o objeto é chamado
        self.model = self.model_builder();

    #construção da rede neural
    def model_builder(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))
        model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear')) #linear retorna todos os voleres dos neurônios
        model.compile(loss='mse', optimizer = tf.keras.optimizers.Adam(learning_rate=0.001))
        return model

    #função que irá fazer toda a negociação, ou seja, as ações que serão tomadas
    def trade(self, state):
        #função que irá tomar uma decisão aleatória no ambiente caso o valor passado pela função random for menor que a de epsilon, possibilitando a exploração do ambiente
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)

        #caso a ação tomada não for aleatória, ou seja, não acionando a condicional anterior, os valores serão buscados por meio da rede neural
        actions = self.model.predict(state)
        #retorna o maior valor dita pela neural
        return np.argmax(actions[0])
    
    #seleciona um grupo de dados para fazer o treinamento
    def batch_train(self, batch_size):
        batch = []

        #pegando somente as amostra do final da memória. Pois fica sem sentido, no caso das ações, pegar uma ação aleatória de 2005 e usar para prever outra em 2020
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])

        for state, action, reward, next_state, done in batch:
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(np.asarray(next_state))[0]) #pega os valores de Q para o prox estado

            #pegando os valores de Q para o estado atual
            target = self.model.predict(state)
            #Adicionando uma recompensa para a ação, onde a coluna [action] recebe o valor dessa recompensa.
            target[0][action] = reward

            self.model.fit(state, target, epochs=1, verbose = 0)

        #Atualizando o epislon, caso ele for maior que o [epsilon_final]
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay


Equação de belmam utilizada para calcular a recompensa

# $$V(s) = max_{a}(R(s, a)) + \gamma V(s')) | $$

# $$\gamma = 0.9 $$


Parte utilizada na recompensa consiste:
# $$ R(s, a)) + \gamma V(s') $$

onde $R(s, a)$ = reward = recompensa no estado atual;

$ \gamma V(s') $ = self.gamma * np.amax(self.model.predict(next_state)[0])

$ s' $ = proximo estado.


#### Sigmoid

## $ y = \frac{1}{1 + e^{-x}} $

In [4]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [5]:
def stocks_price_format(n):
    if n < 0:
        #caso for um valor negativo, adiciona o menos antes do preço($)
        return "- $ {0:2f}".format(abs(n))
    else:
        #se não fica sem o menos
        return "$ {0:2f}".format(abs(n))

In [6]:
dataset = data_reader.DataReader("AAPL", data_source = "yahoo")

In [7]:
dataset.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-09-06,27.075001,26.877501,26.9750,26.924999,107521600.0,25.251354
2016-09-07,27.190001,26.767500,26.9575,27.090000,169457200.0,25.406099
2016-09-08,26.817499,26.309999,26.8125,26.379999,212008000.0,24.740232
2016-09-09,26.430000,25.782499,26.1600,25.782499,186228000.0,24.179869
2016-09-12,26.430000,25.632500,25.6625,26.360001,181171200.0,24.721476


In [8]:
str(dataset.index[0]).split()[0]

'2016-09-06'

In [9]:
def dataset_loader(stock_name):
    #lendo o dataset, onde busca-se pela ação [stock_name] na base de dados do yahoo
    dataset = data_reader.DataReader(stock_name, data_source = "yahoo")
    #data de inicio da ação. Pode ser também em um periodo específico
    start_date = str(dataset.index[0]).split()[0]
    #ultima data da ação coletada
    end_date = str(dataset.index[-1]).split()[0]
    #preço de fechamento da ação no dia
    close = dataset['Close']
    return close

In [10]:
def state_creator(data, timestep, window_size):
    """
    Função onde pega os dados da base de dados de acordo com o timestep (episodios do treinamento)
    e widow_size (tamanho do janelamento dos dados)
    
    """
    starting_id = timestep - window_size + 1
  
    if starting_id >= 0:
        # se o indice da ação for maior do que 0, então ele pega os valores de acordo com o window_size
        windowed_data = data[starting_id:timestep + 1]
    else:
        # se caso o index for 0, ele replica o mesmo valor de acordo com o tamanho do window_size
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
    
    state = []
    for i in range(window_size - 1):
        # normalização dos valores atraves da função da sigmoid = valores entre 0 e 1
        state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
    
    return np.array([state])

In [11]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [12]:
state_creator(data, 20, 5)

array([[0.39114563, 0.55416182, 0.46692298, 0.52996426]])

In [13]:
window_size = 10
episodes = 10
batch_size = 32
data_samples = len(data) - 1

In [14]:
data_samples

1258

In [15]:
trader = IA_Trader(window_size)

In [16]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [18]:
for episode in range(1, episodes + 1):
    print("Episode: {}/{}".format(episode, episodes))
    #criando os estados com 10 valores, sendo o [window_size =- 1], então para 10 valores adiciona +1
    # começando com a primeira posição do dataset = 0   
    state = state_creator(data, 0, window_size + 1)
    #total de lucros com a compra
    total_profit = 0
    #armazena todas as ações que será comprada
    trader.inventory = []

    # percorrendo a base de dados 
    for t in tqdm_notebook(range(data_samples)):
        # ação que será tomada
        action = trader.trade(state)
        # cálculo da próxima ação
        next_state = state_creator(data, t + 1, window_size + 1)
        # inicialização das recompensas
        reward = 0

        # Comprando uma ação se for 1
        if action == 1: 
            # guardando os valores  
            trader.inventory.append(data[t])
            print("AI Trader bought: ", stocks_price_format(data[t]))

        # Vendendo acao se for 2 e verifica se já foi comprado alguma ação
        elif action == 2 and len(trader.inventory) > 0:
            # retira a ação e passa para a variavel  
            buy_price = trader.inventory.pop(0)
            
            # calculo da recompensa, onde é definida pelo valor da venda menos o valor comprado, se for negativo, retorna 0 = sem recompensa
            reward = max(data[t] - buy_price, 0)
            # cálculo do lucro  
            total_profit += data[t] - buy_price
            print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))
            
        # significa que está no ultimo dado da base de dado
        if t == data_samples - 1:
            done = True
        else:
            done = False

        # armazenando na memoria os dados
        trader.memory.append((state, action, reward, next_state, done))

        # passando para o proximo estado
        state = next_state

        if done:
            print("########################")
            print("Total profit: {}".format(total_profit))
            print("########################")

        # quando o tamanho da memoria for maior que o batch size, faz-se o treinamento do modelo
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
     
    # salvando o modelo a cada 10 episodios
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))
    

Episode: 1/10


<ipython-input-18-370a49742593>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for t in tqdm_notebook(range(data_samples)):


  0%|          | 0/1258 [00:00<?, ?it/s]

AI Trader bought:  $ 26.924999
AI Trader sold:  $ 27.090000  Profit: $ 0.165001
AI Trader bought:  $ 26.360001
AI Trader bought:  $ 28.892500
AI Trader sold:  $ 28.395000  Profit: $ 2.035000
AI Trader bought:  $ 28.392500
AI Trader bought:  $ 28.387501
AI Trader sold:  $ 28.655001  Profit: - $ 0.237499
AI Trader bought:  $ 28.177500
AI Trader sold:  $ 28.129999  Profit: - $ 0.262501
AI Trader bought:  $ 28.250000
AI Trader bought:  $ 28.262501
AI Trader sold:  $ 28.472500  Profit: $ 0.084999
AI Trader sold:  $ 28.514999  Profit: $ 0.337500
AI Trader sold:  $ 29.012501  Profit: $ 0.762501
AI Trader bought:  $ 29.075001
AI Trader bought:  $ 29.334999
AI Trader bought:  $ 29.245001
AI Trader sold:  $ 29.367500  Profit: $ 1.105000
AI Trader bought:  $ 29.264999
AI Trader bought:  $ 29.412500
AI Trader sold:  $ 29.562500  Profit: $ 0.487499
AI Trader sold:  $ 28.620001  Profit: - $ 0.714998
AI Trader bought:  $ 28.430000
AI Trader bought:  $ 27.897499
AI Trader bought:  $ 27.457500
AI Trade

KeyboardInterrupt: 